In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler # импортируем библиотеку

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from datetime import datetime, date, time
from sklearn import preprocessing
from sklearn import utils
#импорт данных
df_match=pd.read_csv('match_final.csv')
df_match['date'] = pd.to_datetime(df_match['date'],dayfirst=True).dt.strftime('%d-%m-%Y')
df_match = df_match.dropna()
df_match = df_match.reset_index(drop=True)
df_match.head(20)

,league_id,name,season,stage,date,match_api_id,home_team_api_id,home_team,away_team_api_id,away_team,...,WHH,WHD,WHA,VCH,VCD,VCA,home_result,away_result,home_superiority,away_superiority
0,1,Belgium Jupiler League,2015/2016,1,25-07-2015,1979833,8571,KV Kortrijk,9985,Standard de Liège,...,2.50,3.25,2.62,2.70,3.40,2.63,1,0,0,0
1,1,Belgium Jupiler League,2015/2016,1,25-07-2015,1979835,8573,KV Oostende,8203,KV Mechelen,...,2.45,3.10,2.80,2.50,3.30,2.88,1,0,0,0
2,1,Belgium Jupiler League,2015/2016,1,25-07-2015,1979836,10000,SV Zulte-Waregem,9994,Sporting Lokeren,...,2.30,3.25,2.88,2.30,3.40,3.10,1,0,0,0
3,1,Belgium Jupiler League,2015/2016,1,26-07-2015,1979837,10001,KVC Westerlo,9991,KAA Gent,...,4.00,3.50,1.80,4.10,3.70,1.87,1,1,0,0
4,1,Belgium Jupiler League,2015/2016,1,26-07-2015,1979838,8635,RSC Anderlecht,8475,Waasland-Beveren,...,1.20,6.00,11.00,1.20,6.50,15.00,1,0,0,0
5,1,Belgium Jupiler League,2015/2016,1,26-07-2015,1979839,9986,Sporting Charleroi,274581,Royal Excel Mouscron,...,1.70,3.40,4.80,1.65,3.80,5.50,1,0,0,0
6,1,Belgium Jupiler League,2015/2016,10,04-10-2015,1979896,8635,RSC Anderlecht,8203,KV Mechelen,...,1.40,4.33,7.00,1.44,4.60,7.00,1,1,5,1
7,1,Belgium Jupiler League,2015/2016,10,04-10-2015,1979897,9991,KAA Gent,8342,Club Brugge KV,...,2.10,3.40,3.10,2.15,3.50,3.30,1,0,4,2
8,1,Belgium Jupiler League,2015/2016,10,04-10-2015,1979898,9987,KRC Genk,9985,Standard de Liège,...,2.00,3.50,3.30,2.15,3.60,3.25,1,0,4,1
9,1,Belgium Jupiler League,2015/2016,10,02-10-2015,1979899,8573,KV Oostende,9994,Sporting Lokeren,...,1.91,3.50,3.60,1.87,3.70,4.10,1,0,4,2


In [2]:
#сила команд на каждый матч
df_match['ht_strength']= df_match['home_player_1']+df_match['home_player_2']+df_match['home_player_3']+df_match['home_player_4']+df_match['home_player_5']+df_match['home_player_6']+df_match['home_player_7']+df_match['home_player_8']+df_match['home_player_9']+df_match['home_player_10']+df_match['home_player_11']
df_match['at_strength']= df_match['away_player_1']+df_match['away_player_2']+df_match['away_player_3']+df_match['away_player_4']+df_match['away_player_5']+df_match['away_player_6']+df_match['away_player_7']+df_match['away_player_8']+df_match['away_player_9']+df_match['away_player_10']+df_match['away_player_11']


In [3]:
#создаю колонку с результатами матча
# исход 1 - выигрыш home_team
# исход 0 - прогрыш away_team
# исход 0.5 - ничья 
df_match['result_home_team_api_id'] = df_match.apply(lambda x: 1 if x['home_team_goal'] > x['away_team_goal'] else (0 if x['home_team_goal'] < x['away_team_goal'] else 0.5), axis=1)
df_match['result_away_team_api_id'] = df_match.apply(lambda x: 1 if x['home_team_goal'] < x['away_team_goal'] else (0 if x['home_team_goal'] > x['away_team_goal'] else 0.5), axis=1)
df_match['result_home_team_api_id']=df_match['result_home_team_api_id'].astype('int')
df_match['result_away_team_api_id']=df_match['result_away_team_api_id'].astype('int')

display(df_match)

,league_id,name,season,stage,date,match_api_id,home_team_api_id,home_team,away_team_api_id,away_team,...,VCD,VCA,home_result,away_result,home_superiority,away_superiority,ht_strength,at_strength,result_home_team_api_id,result_away_team_api_id
0,1,Belgium Jupiler League,2015/2016,1,25-07-2015,1979833,8571,KV Kortrijk,9985,Standard de Liège,...,3.4,2.63,1,0,0,0,754,776,1,0
1,1,Belgium Jupiler League,2015/2016,1,25-07-2015,1979835,8573,KV Oostende,8203,KV Mechelen,...,3.3,2.88,1,0,0,0,776,762,1,0
2,1,Belgium Jupiler League,2015/2016,1,25-07-2015,1979836,10000,SV Zulte-Waregem,9994,Sporting Lokeren,...,3.4,3.10,1,0,0,0,761,754,1,0
3,1,Belgium Jupiler League,2015/2016,1,26-07-2015,1979837,10001,KVC Westerlo,9991,KAA Gent,...,3.7,1.87,1,1,0,0,739,805,1,0
4,1,Belgium Jupiler League,2015/2016,1,26-07-2015,1979838,8635,RSC Anderlecht,8475,Waasland-Beveren,...,6.5,15.00,1,0,0,0,818,734,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,21518,Spain LIGA BBVA,2015/2016,9,25-10-2015,2030167,9906,Atlético Madrid,10267,Valencia CF,...,4.0,7.00,1,0,3,2,903,863,1,0
2452,21518,Spain LIGA BBVA,2015/2016,9,24-10-2015,2030168,9864,Málaga CF,9783,RC Deportivo de La Coruña,...,3.4,3.40,1,0,3,4,830,833,1,0
2453,21518,Spain LIGA BBVA,2015/2016,9,26-10-2015,2030169,8315,Athletic Club de Bilbao,9869,Real Sporting de Gijón,...,4.2,7.00,1,0,2,3,875,811,1,0
2454,21518,Spain LIGA BBVA,2015/2016,9,24-10-2015,2030170,7878,Granada CF,8603,Real Betis Balompié,...,3.4,3.30,1,1,1,3,810,828,1,0


In [4]:

# Для домашних команд
home_teams = df_match[['home_team_api_id', 'ht_strength', 'home_superiority','result_home_team_api_id']].copy()
home_teams.columns = ['team_api_id', 'team_strength', 'team_superiority','result']

# Для гостевых команд
away_teams = df_match[['away_team_api_id', 'at_strength', 'away_superiority','result_away_team_api_id']].copy()
away_teams.columns = ['team_api_id', 'team_strength', 'team_superiority','result']

# Объединяем домашние и гостевые команды
combined_df = pd.concat([home_teams, away_teams], ignore_index=True)

# Теперь у нас есть таблица с тремя столбцами: 'team_api_id', 'team_strength', и 'team_superiority'
display(combined_df)

,team_api_id,team_strength,team_superiority,result
0,8571,754,0,1
1,8573,776,0,1
2,10000,761,0,1
3,10001,739,0,1
4,8635,818,0,1
...,...,...,...,...
4907,10267,863,2,0
4908,9783,833,4,0
4909,9869,811,3,0
4910,8603,828,3,0


In [5]:
import pandas as pd
from itertools import combinations

# Предполагаем, что combined_df - это ваш объединенный DataFrame
# Создаем все возможные уникальные пары
team_pairs = list(combinations(combined_df['team_api_id'].unique(), 2))

# Пустой список для хранения данных каждой пары
pairs_data = []

# Заполняем список данными
for team1, team2 in team_pairs:
    team1_data = combined_df[combined_df['team_api_id'] == team1].iloc[0]
    team2_data = combined_df[combined_df['team_api_id'] == team2].iloc[0]
    pairs_data.append({
        'team_id_side_1': team1,
        'team_id_side_2': team2,
        'strength_side_1': team1_data['team_strength'],
        'strength_side_2': team2_data['team_strength'],
        'superiority_side_1': team1_data['team_superiority'],
        'superiority_side_2': team2_data['team_superiority'],
        'result_side_1': team1_data['result'],
        'result_side_2': team1_data['result'],
    })

# Создаем DataFrame из списка пар
pairs_df = pd.DataFrame(pairs_data)

display(pairs_df)

,team_id_side_1,team_id_side_2,strength_side_1,strength_side_2,superiority_side_1,superiority_side_2,result_side_1,result_side_2
0,8571,8573,754,776,0,0,1,1
1,8571,10000,754,761,0,0,1,1
2,8571,10001,754,739,0,0,1,1
3,8571,8635,754,818,0,0,1,1
4,8571,9986,754,778,0,0,1,1
...,...,...,...,...,...,...,...,...
11776,8634,8306,924,804,5,3,1,1
11777,8634,9910,924,853,5,4,1,1
11778,8302,8306,875,804,3,3,1,1
11779,8302,9910,875,853,3,4,1,1


In [6]:
df_match = pairs_df.copy()

In [11]:
#нормализация
#df_match['ht_strength']=(df_match['ht_strength']-df_match['ht_strength'].min())/(df_match['ht_strength'].max()-df_match['ht_strength'].min())
#df_match['at_strength']=(df_match['at_strength']-df_match['at_strength'].min())/(df_match['at_strength'].max()-df_match['at_strength'].min())
# #стандартизация
# df_x = df_match[['strength_side_1', 'strength_side_2', 'superiority_side_1','superiority_side_2']]
# df_match[['strength_side_1', 'strength_side_2',  'superiority_side_1','superiority_side_2']] = (df_x-df_x.mean())/df_x. std()
# #display(df_match)

In [12]:
# df_match['result_side_1']

In [7]:
df_match.columns = df_match.columns.str.replace(' ', '')
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11781 entries, 0 to 11780
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   team_id_side_1      11781 non-null  int64
 1   team_id_side_2      11781 non-null  int64
 2   strength_side_1     11781 non-null  int64
 3   strength_side_2     11781 non-null  int64
 4   superiority_side_1  11781 non-null  int64
 5   superiority_side_2  11781 non-null  int64
 6   result_side_1       11781 non-null  int64
 7   result_side_2       11781 non-null  int64
dtypes: int64(8)
memory usage: 736.4 KB


In [8]:
df_match.columns.values.tolist ()

['team_id_side_1',
 'team_id_side_2',
 'strength_side_1',
 'strength_side_2',
 'superiority_side_1',
 'superiority_side_2',
 'result_side_1',
 'result_side_2']

In [10]:
#анализ для away_team

log_reg = LogisticRegression()
X = df_match[['strength_side_1', 'strength_side_2', 'superiority_side_1','superiority_side_2']]
Y=df_match['result_side_1']
lab = preprocessing.LabelEncoder()
Y = lab.fit_transform(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)#делим не 70/30, а случайные куски

log_reg.fit(X_train, Y_train)
Y_predicted = log_reg.predict(X_test)


print('acc:%.2f'% accuracy_score(Y_test,Y_predicted))
print('prec:%.2f'% precision_score(Y_test,Y_predicted)) 
print('rec:%.2f' % recall_score(Y_test,Y_predicted)) 
print(log_reg.coef_[0][0])
#y=1/(1+е^(-z))
#Z = b0+b1x1+....
print(log_reg.coef_)
print(log_reg.intercept_)

acc:0.62
prec:0.63
rec:0.97
0.004590691494500416
[[ 0.00459069 -0.00400949  0.02761353 -0.00443886]]
[-0.00474593]


In [16]:
print(log_reg.intercept_[0])
Z=log_reg.intercept_[0]+log_reg.coef_[0][0]*df_match['strength_side_1']+log_reg.coef_[0][1]*df_match['strength_side_2']+log_reg.coef_[0][2]*df_match['superiority_side_1']+log_reg.coef_[0][3]*df_match['superiority_side_2']
df_match['prediction']=1/(1+np.exp(-Z))
df_match['prediction'].min()

-0.004745928263107228


0.35000551340398484

In [19]:
df = df_match
team = pd.read_csv("team и team_attributes (1).csv")

# Переименование столбца для совпадения с первым датафреймом.
team.rename(columns={'team_api_id': 'team_id_side_1'}, inplace=True)

# Выполняем слияние двух таблиц с использованием нового названия столбца `team_id_side_1`
result = df.merge(team[['team_id_side_1', 'team_long_name']], left_on='team_id_side_1', right_on='team_id_side_1', how='left')


# Еще одна переименовка для слияния таблиц
team.rename(columns={'team_id_side_1': 'team_id_side_2'}, inplace=True)
# Выполняем слияние с использованием нового названия столбца `team_id_side_2`
final = result.merge(team[['team_id_side_2', 'team_long_name']], left_on='team_id_side_2', right_on='team_id_side_2', how='left')


# Посмотрим результат
display(final)

,team_id_side_1,team_id_side_2,strength_side_1,strength_side_2,superiority_side_1,superiority_side_2,result_side_1,result_side_2,prediction,team_long_name_x,team_long_name_y
0,8571,8573,754,776,0,0,1,1,0.585470,KV Kortrijk,KV Oostende
1,8571,8573,754,776,0,0,1,1,0.585470,KV Kortrijk,KV Oostende
2,8571,8573,754,776,0,0,1,1,0.585470,KV Kortrijk,KV Oostende
3,8571,8573,754,776,0,0,1,1,0.585470,KV Kortrijk,KV Oostende
4,8571,8573,754,776,0,0,1,1,0.585470,KV Kortrijk,KV Oostende
...,...,...,...,...,...,...,...,...,...,...,...
346214,8306,9910,804,853,3,4,1,1,0.582041,UD Las Palmas,RC Celta de Vigo
346215,8306,9910,804,853,3,4,1,1,0.582041,UD Las Palmas,RC Celta de Vigo
346216,8306,9910,804,853,3,4,1,1,0.582041,UD Las Palmas,RC Celta de Vigo
346217,8306,9910,804,853,3,4,1,1,0.582041,UD Las Palmas,RC Celta de Vigo


In [20]:
# Cохраним итоговый файл
final = pd.read_csv("prediction following match.csv")